In [1]:
from parameters import *
from functions import *
from PDE_solutions import *
walks = 500
N = 200

In [2]:
X0

[-1.5, 1.0]

# MLMV



In [3]:
def CI(mean, std, N, confidence):
    ''' Compute the confidence interval at the desired confidence level 
    '''
    alfa = 1 - confidence
    C_alfa2 = st.t.ppf(1-alfa/2,N-1)
    lowerB = mean - C_alfa2*std/np.sqrt(N)
    upperB = mean + C_alfa2*std/np.sqrt(N)
    return lowerB, upperB


In [4]:
def OneLevelRandomWalk(X0, Y0, N, T):
    ''' X0: initial position
        N: number of steps
        T: Final time'''
    X = []
    Y = []
    dt = T/N
    sigmaSqrtDt = sigma * np.sqrt(dt)
    X.append(X0)
    Y.append(Y0)
    finalT = dt
    XisIn = False
    YisIn = False
    for i in range(N-1):
        Norm = norm.rvs(size=2)
        X0 = X0 + u(X0) * dt + sigmaSqrtDt* Norm
        Y0 = Y0 + u(Y0) * dt + sigmaSqrtDt* Norm 
        if (not XisIn) : X.append(X0)
        if (not YisIn) : Y.append(Y0)
        finalT = finalT + dt
        rX = np.sqrt( X0[0]**2 + X0[1]**2 )
        rY = np.sqrt( Y0[0]**2 + Y0[1]**2 ) 
        if (rX < R) : XisIn = True
        if (rY < R) : YisIn = True
        if(XisIn and YisIn):
            break
    
    return np.asarray(X), np.asarray(Y), XisIn, YisIn

In [5]:
def OneLevelMonteCarlo(X0, X1, walks0, walks1, N, T = 1, confidence = 0.95, seed = 1, tol = 1e-6,
                    PDEProb = -1, verbose = 2):
    polluted0 = np.zeros(walks0)
    polluted1 = np.zeros(walks1)

    start = time.time()
    for w in range(walks0):
        if (verbose == 2 and w%100 == 0):
            print('Current walk: ', w )
        #_, currentTime = NaiveRandomWalk(X0, N, T)
        #_, currentTime2 = NaiveRandomWalk(X1, N, T)
        _,_,isIn0,isIn1 = OneLevelRandomWalk(X0,X1,N,T)
        if isIn0:
            polluted0[w] = 1
        if isIn1:
            polluted1[w] = 1
    for w in range(walks0,walks1):
        _,_,isIn0,isIn1 = OneLevelRandomWalk(X0,X1,N,T)
        if isIn1:
            polluted1[w] = 1
    end = time.time()

    mean0 = polluted0.mean()
    mean1 = polluted1.mean()
    mean = mean1 + np.mean(polluted0-polluted1[range(walks0)])

    Var_1 = np.std(polluted1)**2 / walks1 
    Var_0 = np.std(polluted0)**2 / walks0
    Var_mix = np.std(polluted0-polluted1[range(walks0)])**2/walks0
    Var = Var_1 + Var_mix
    if verbose >=1:
        print(f'\nNumber of simulations: %d. Time needed = %.2f s' % (walks, end-start))
        print(f'Estimated variance: {Var}' % (Var))
        print(f'The estimated probability at {X0} is: {mean} (using MC)')
        if PDEProb != -1:
            print(f'\nPDE result at {X0} is:  {PDEProb}')
    return mean,Var, Var_0, Var_1, Var_mix, walks0, walks1

In [6]:
def MultilevelFunctionForLDifferentTimesSteps(X_0,N,T,L):
    ''' X_0: initial position
    N: vector of the number of steps
    T: Final time
    L : Level to which we are interested walks on L elements of X'''
    dt = T/N
    #print("dt :", dt)
    sigmaSqrtDt = sigma * np.sqrt(dt)
    #print("sigmasqrtdt :", sigmaSqrtDt)
    finalT = dt
    areIn = np.full(len(N), False)
    #print("are In . ", areIn)
    X = np.outer(X_0,np.ones(L+1)).T
    #print("X", X)

    for i in range(N[L]-1):
        Norm = norm.rvs(size=2)
        
        for l in range(L+1):
            if ((not areIn[l]) and (finalT[l] < T)) :
                X[l] = X[l] + u(X[l]) * dt[l] + sigmaSqrtDt[l]* Norm
                r = np.sqrt( X[l,0]**2 + X[l,1]**2 ) 
                if (r < R) : areIn[l] = True
            
        finalT = finalT + dt
        if(areIn.sum() == L+1):
            break
    
    return areIn

In [7]:
def MultilevelFunctionForLDifferentPositions(X_0,N,T,L):
    ''' X: initial positions
    N: number of steps
    T: Final time
    L : Level to which we are interested walks on L elements of X'''
    dt = T/N
    sigmaSqrtDt = sigma * np.sqrt(dt)
    finalT = dt
    areIn = np.full(np.shape(X_0)[0], False)
    X = np.array(X_0)

    for i in range(N-1):
        Norm = norm.rvs(size=2)
        
        for l in range(L+1):
            if (not areIn[l]) :
                X[l] = X[l] + u(X[l]) * dt + sigmaSqrtDt* Norm
                r = np.sqrt( X[l,0]**2 + X[l,1]**2 ) 
                if (r < R) : areIn[l] = True
            
        finalT = finalT + dt
        if(areIn.sum() == L+1):
            break
    
    return areIn

In [8]:
def MultiLevelMonteCarlo(L, X0, Walks, Functions, N, T = 1, confidence = 0.95, seed = 1, tol = 1e-6,
                    PDEProb = -1, verbose = 2):
    """
    Input:
        L is the number of levels, where L will be the level we want to estimate with 0 being the level with smallest variance
        Walks is a vector of size L+1, where Walks[l] is the number of walks for the level l
        X0 is a L+1 by 2 matrix storing the starting points, X[l,:] being the starting point of the l-th level
        Functions : the f to apply for the l-th walk returns a l size vector
    """
    
    #E stores the expectation of each level E[l] = E[Pl-P(l-1)] and E[0] = E[P0]
    E = np.zeros(L+1)
    VAR = np.zeros(L+1)
    
    #stores the polluted values, L lines
    polluted = np.empty((L+1,Walks[0]))
    Walks.append(0)
    for l in range(L,0,-1):
        if(verbose == 2) : print('Calculating level', l)

        for w in range(Walks[l+1],Walks[l]):
            areInR = Functions(X0, N, T,l)
            polluted[:,w] = areInR 
        
        E[l] = np.mean(polluted[l,0:Walks[l]] - polluted[l-1,0:Walks[l]])
        VAR[l] = np.std(polluted[l,0:Walks[l]] - polluted[l-1,0:Walks[l]])**2 / Walks[l]
    
    #runs the P0 walk
    if(verbose == 2) : print('Calculating level 0')
    for w in range(Walks[1],Walks[0]):
        areInR = Functions(X0, N, T,0)
        polluted[:,w] = areInR 
        
    E[0] = np.mean(polluted[0,:])
    VAR[0] = np.std(polluted[0,:])**2 / Walks[0]
    Var = np.sum(VAR)
    mean = np.sum(E)
    Var0 = np.std(polluted[L,0:Walks[L]])**2 / Walks[L]
    
    if verbose >=1:
        #print(f'\nNumber of simulations: %d. Time needed = %.2f s' % (walks, end-start))
        print(f'The estimated probability at {X0} is: {mean} (using MC)')
        print('with the variance : ', Var)
        print('Whithout the variance reduction ', Var0)
        #if PDEProb != -1:
            #print(f'\nPDE result at {X0} is:  {PDEProb}')
            
    return E, VAR, Var, mean, Var0

# Check if there is some POSITIVE correlation (heuristic)

In [ ]:
for walks in [10,50,100,500,1000]:
    X0 = [-1.5,1]
    X1 = [-1.4,1]
    print(OneLevelMonteCarlo(X0,X1, walks, walks*3, N, verbose=0))

(0.6333333333333333, 0.007740740740740739, 0.024, 0.007740740740740739, 0.0, 10, 30)
(0.5866666666666667, 0.001982814814814814, 0.004872, 0.0015908148148148142, 0.00039199999999999993, 50, 150)
(0.6866666666666666, 0.0010034074074074073, 0.0022440000000000003, 0.0007044074074074074, 0.000299, 100, 300)


In [40]:
L = 2
N = 200
X0 = np.array([[-1.1,1],
   [-1.3,1],
   [-1.5,1]])
Walks = [2000,1000,200]
Functions = MultilevelFunctionForLDifferentPositions
E, VAR, Var, mean, VarFromTheLwalk = MultiLevelMonteCarlo(L,X0,Walks, Functions, N)

Calculating level 2
Calculating level 1
Calculating level 0
The estimated probability at [-1.5  1. ] is: 0.6715 (using MC)
with the variance :  0.0004045908749999999
Whithout the variance reduction  0.0010398749999999998


In [52]:
E, VAR, Var, mean, VarFromTheLwalk

(array([ 0.715, -0.019, -0.02 ]),
 array([6.788e-05, 5.464e-05, 3.480e-04]),
 0.000470516185185185,
 0.6763333333333333,
 0.0009855)

In [22]:
BasicMonteCarlo(X0,1000,200)

Current walk:  0
Current walk:  100
Current walk:  200
Current walk:  300
Current walk:  400
Current walk:  500
Current walk:  600
Current walk:  700
Current walk:  800
Current walk:  900

Number of simulations: 1000. Time needed = 4.60 s
Estimated variance: 0.4606378047741926
The estimated probability at [-1.5  1. ] is: 0.695 (using MC)
Confidence interval: [ 0.695 +- 0.028584734169919912 ]	with P = 0.95%


(0.695, 0.4606378047741926, 0.66641526583008, 0.7235847341699199)

In [16]:
L = 5
X0 = np.array([-1.5,1])

N0 = 50
NL = 200
N = createVectorN(N0,NL,L)
Walks = [50000, 40000, 30000, 20000, 10000, 1000]
Function = MultilevelFunctionForLDifferentTimesSteps
E, VAR, Var, mean, VarFromTheLwalk = MultiLevelMonteCarlo(L,X0,Walks, Function, N)

Calculating level 5
Calculating level 4
Calculating level 3
Calculating level 2
Calculating level 1
Calculating level 0
The estimated probability at [-1.5  1. ] is: 0.6951766666666667 (using MC)
with the variance :  0.00011236324652596297
Whithout the variance reduction  0.00021039900000000003


In [ ]:
### X = np.outer(X_0,np.ones(L+1)).T
X

In [ ]:
t0 = 0.1
tf = 0.01
L = 3
M = int( (t0/tf)**(1/L) )
M